In [68]:

from dataclasses import dataclass

@dataclass
class Parameters():
    batch_size: int = 48
    epochs: int = 1
    verbosity: int = 1
    step_size: int = 374
    number_folds: int = 2
    

In [70]:
# Template
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import KFold
import numpy as np
import dataclasses

# Load data and preprocess
x_train = ...
x_test = ...
y_train = ...
y_test = ...

# Load data and preprocess
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x_train.reshape((x_train.shape[0], 28, 28, 1)) / 255.0
x_test = x_test.reshape((x_test.shape[0], 28, 28, 1)) / 255.0
y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)


    

# Something like this as first Model
def create_model_1():
    model = keras.Sequential([
        layers.Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1)),
        layers.MaxPooling2D((2,2)),
        layers.Flatten(),
        layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Something like this as second Model
def create_model_2():
    model = keras.Sequential([
        layers.Conv2D(64, (3,3), activation='relu', input_shape=(28,28,1)),
        layers.MaxPooling2D((2,2)),
        layers.Conv2D(64, (3,3), activation='relu'),
        layers.Flatten(),
        layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Something like this as Third Model
def create_model_3():
    model = keras.Sequential([
        layers.Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1)),
        layers.Conv2D(64, (3,3), activation='relu'),
        layers.MaxPooling2D((2,2)),
        layers.Conv2D(64, (3,3), activation='relu'),
        layers.Flatten(),
        layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Perform cross-validation
models = [create_model_1(), create_model_2(), create_model_3()]
num_folds = Parameters.number_folds
kfold = KFold(n_splits=num_folds, shuffle=True)
fold_acc_scores = []
for i, (train, test) in enumerate(kfold.split(x_train, y_train)):
    print('Fold', i+1)
    train_x, train_y = x_train[train], y_train[train]
    test_x, test_y = x_train[test], y_train[test]
    for j, model in enumerate(models):
        print('Model', j+1)
        history = model.fit(train_x, train_y, epochs=Parameters.epochs, batch_size=Parameters.batch_size, validation_data=(test_x, test_y), verbose=Parameters.verbosity, steps_per_epoch=Parameters.step_size)
        _, acc = model.evaluate(test_x, test_y, verbose=Parameters.verbosity)
        print('Validation accuracy:', acc)
        fold_acc_scores.append((i, j, acc))

# Find the best model
best_model_index = np.argmax([score[2] for score in fold_acc_scores])
print(fold_acc_scores)
print(best_model_index)
best_model = models[fold_acc_scores[best_model_index][1]]
print(fold_acc_scores)
print(best_model_index)
print(best_model)
print('Best model is model', best_model_index+1)

# Evaluate the best model on the test set
test_loss, test_acc = best_model.evaluate(x_test, y_test)
print('Test accuracy:', test_acc)

Fold 1
Model 1
938/938 [==============================] - 3s 4ms/step - loss: 0.2276 - accuracy: 0.9339
Validation accuracy: 0.9338666796684265
Model 2
938/938 [==============================] - 8s 8ms/step - loss: 0.1036 - accuracy: 0.9689
Validation accuracy: 0.9688666462898254
Model 3
938/938 [==============================] - 11s 12ms/step - loss: 0.0893 - accuracy: 0.9731
Validation accuracy: 0.9730666875839233
Fold 2
Model 1
938/938 [==============================] - 3s 3ms/step - loss: 0.1352 - accuracy: 0.9612
Validation accuracy: 0.9612333178520203
Model 2
938/938 [==============================] - 7s 8ms/step - loss: 0.0819 - accuracy: 0.9748
Validation accuracy: 0.9747666716575623
Model 3
938/938 [==============================] - 11s 12ms/step - loss: 0.0661 - accuracy: 0.9794
Validation accuracy: 0.9794333577156067
[(0, 0, 0.9338666796684265), (0, 1, 0.9688666462898254), (0, 2, 0.9730666875839233), (1, 0, 0.9612333178520203), (1, 1, 0.9747666716575623), (1, 2, 0.9794333577

In [ ]:
# Create a Baseline Model

In [ ]:
# Compare Baseline vs best_model

In [ ]:
# plot graph for learning curve and loss curve from baseline and best_model

In [72]:
# summarise best model
best_model.summary()


Model: "sequential_115"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_183 (Conv2D)         (None, 26, 26, 32)        320       
                                                                 
 conv2d_184 (Conv2D)         (None, 24, 24, 64)        18496     
                                                                 
 max_pooling2d_107 (MaxPooli  (None, 12, 12, 64)       0         
 ng2D)                                                           
                                                                 
 conv2d_185 (Conv2D)         (None, 10, 10, 64)        36928     
                                                                 
 flatten_92 (Flatten)        (None, 6400)              0         
                                                                 
 dense_166 (Dense)           (None, 10)                64010     
                                                    

In [ ]:
# Export Model 
!mkdir -p saved_model
best_model.save('saved_model/sensor_model')
best_model.save('saved_model/sensor_model.h5')

In [ ]:
# Save the weight for the Js Model
best_model.save_weights('./checkpoints/my_checkpoint')
